# Numpy

## Intro

本课将介绍通过 Python 有效导入、存储和操作内存数据的主要技巧。这个主题非常广泛, 因为数据集的来源与格式都十分丰富, 比如文档集合、图像集合、声音片段集合、数值数据集合, 等等。这些数据虽然存在明显的异构性, 但是将所有数据简单地看作数字数组非常有助于我们理解和处理数据。
例如, 可以将图像（尤其是数字图像）简单地看作二维数字数组, 这些数字数组代表各区 域的像素值; 声音片段可以看作时间和强度的一维数组; 文本也可以通过各种方式转换成 数值表示, 一种可能的转换是用二进制数表示特定单词或单词对出现的频率。不管数据是何种形式, 第一步都是将这些数据转换成数值数组形式的可分析数据。
正因如此, 有效地存储和操作数值数组是数据科学中绝对的基础过程。我们将详细介绍 NumPy。NumPy（Numerical Python 的简称）提供了高效存储和操作密集数据缓存的接口。在某些方面, NumPy 数组与 Python 内置的列表类型非常相似。但是随着数组在维度上变大, NumPy 数组提供了更加高效的存储和数据操作。NumPy 数组几乎是整个 Python 数据科学工具生态系统的核心。因此, 不管你对数据科学的哪个方面感兴趣, 花点时间学习如何有效地使用 NumPy 都是非常值得的。
如果你听从前言给出的建议安装了 Anaconda, 那么你已经安装好 NumPy, 并可以使用它 了。如果你是个体验派, 则可以到 NumPy 网站（http://www.numpy.org/）按照其安装指导进行安装。安装好后, 你可以导入 NumPy 并再次核实你的 NumPy 版本:

In [13]:
import numpy as np
import matplotlib.pyplot as plt
np.__version__

'1.22.4'

## Python 数据类型

要实现高效的数据驱动科学和计算，需要理解数据是如何被存储和操作的。本节将介绍在
Python 语言中数据数组是如何被处理的，并对比 NumPy 所做的改进。理解这个不同之处
是理解本书其他内容的基础。
Python 的用户往往被其易用性所吸引，其中一个易用之处就在于动态输入。静态类型的语
言（如 C 或 Java）往往需要每一个变量都明确地声明，而动态类型的语言（例如 Python）
可以跳过这个特殊规定。例如在 C 语言中，你可能会按照如下方式指定一个特殊的操作：

``` c
/* C代码 */
int result = 0;
for(int i=0; i<100; i++){
 result += i;
}
```
而在 Python 中，同等的操作可以按照如下方式实现：

``` Python
# Python代码
result = 0
for i in range(100):
 result += i
```
注意这里最大的不同之处：在 C 语言中，每个变量的数据类型被明确地声明；而在  Python 中，类型是动态推断的。这意味着可以将任何类型的数据指定给任何变量：
``` Python
# Python代码
x = 4
x = "four"
```

这里已经将 x 变量的内容由整型转变成了字符串，而同样的操作在 C 语言中将会导致（取
决于编译器设置）编译错误或其他未知的后果：

``` C
/* C代码 */
int x = 4;
x = "four"; // 编译失败
```

这种灵活性是使 Python 和其他动态类型的语言更易用的原因之一。理解这一特性如何工作
是学习用 Python 有效且高效地分析数据的重要因素。但是这种类型灵活性也指出了一个事
实：Python 变量不仅是它们的值，还包括了关于值的类型的一些额外信息，本节接下来的
内容将进行更详细的介绍。

### Python 整型不仅仅是一个整型
标准的 Python 实现是用 C 语言编写的。这意味着每一个 Python 对象都是一个聪明的伪 C 语言结构体，该结构体不仅包含其值，还有其他信息。例如，当我们在 Python 中定义一个整型，例如 x = 10000 时，x 并不是一个“原生”整型，而是一个指针，指向一个 C 语言的复合结构体，结构体里包含了一些值。查看 Python 3.4 的源代码，可以发现整型（长整型）的定义，如下所示（C 语言的宏经过扩展之后）：
```C
struct _longobject {
 long ob_refcnt;
 PyTypeObject *ob_type;
 size_t ob_size;
 long ob_digit[1];
};
```
Python 3.4 中的一个整型实际上包括 4 个部分。

- ob_refcnt 是一个引用计数，它帮助 Python 默默地处理内存的分配和回收。
- ob_type 将变量的类型编码。
- ob_size 指定接下来的数据成员的大小。
- ob_digit 包含我们希望 Python 变量表示的实际整型值。

这意味着与 C 语言这样的编译语言中的整型相比，在 Python 中存储一个整型会有一些开
销。

<div align=center>
<img width="500" src="https://zhaochenyang20.github.io/pic/embed/6_29_1.jpg"/>
</div>
<div align=center>C 整型和 Python 整型的区别，这里 PyObject_HEAD 是结构体中包含引用计数、类型编码和其他部分。</div>

两者的差异在于，C 语言整型本质上是对应某个内存位置的标签，里面存储的字节会编码成整型。而 Python 的整型其实是一个指针，指向包含这个 Python 对象所有信息的某个内存位置，其中包括可以转换成整型的字节。由于 Python 的整型结构体里面还包含了大量额外的信息，所以 Python 可以自由、动态地编码。但是，Python 类型中的这些额外信息也会成为负担，在多个对象组合的结构体中尤其明显。

### Python列表不仅仅是一个列表
设想如果使用一个包含很多 Python 对象的 Python 数据结构，会发生什么？ Python 中的标准可变多元素容器是列表。可以用如下方式创建一个整型值列表：

``` Python
In [1]: L = list(range(10))

In [2]: L
Out[2]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
```

或者创建一个字符串列表：

``` Python
In [4]: L2 = [str(each) for each in L]

In [5]: L2
Out[5]: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]: type(L2[0])
Out[6]: str
```

因为 Python 的动态类型特性，甚至可以创建一个异构的列表：

``` Python
In [7]:  L3 = [True, "2", 3.0, 4]

In [8]: [type(item) for item in L3]
Out[8]: [bool, str, float, int]
```

但是想拥有这种灵活性也是要付出一定代价的：为了获得这些灵活的类型，列表中的每一项必须包含各自的类型信息、引用计数和其他信息；也就是说，每一项都是一个完整的 Python 对象。来看一个特殊的例子，如果列表中的所有变量都是同一类型的，那么很多信息都会显得多余——将数据存储在固定类型的数组中应该会更高效。动态类型的列表和固定类型的（NumPy 式）数组间的区别如下图所示。

<div align=center>
<img width="500" src="https://zhaochenyang20.github.io/pic/embed/6_29_2.jpg"/>
</div>
<div align=center>numpy 列表和 Python 列表的区别。</div>

在实现层面，数组基本上包含一个指向连续数据块的指针。另一方面，Python 列表包含一个指向指针块的指针，这其中的每一个指针对应一个完整的 Python 对象（如前面看到的 Python 整型）。另外，列表的优势是灵活，因为每个列表元素是一个包含数据和类型信息的完整结构体，而且列表可以用任意类型的数据填充。固定类型的 NumPy 式数组缺乏这种灵活性，但是能更有效地存储和操作数据。

# 创建和生成

本节主要介绍 array 的创建和生成。为什么会把这个放在最前面呢？主要有以下两个方面原因：

- 在实际工作过程中，我们时不时需要验证或查看 array 相关的 API 或互操作。
- 有时候在使用 sklearn，matplotlib，PyTorch，Tensorflow 等工具时也需要一些简单的数据进行实验。

所以，先学会如何快速拿到一个 array 是有很多益处的。本节我们主要介绍以下几种常用的创建方式：

- 使用列表或元组
- 使用 arange
- 使用 linspace/logspace
- 使用 ones/zeros
- 使用 random
- 从文件读取

其中，最常用的一般是 linspace/logspace 和 random，前者常常用在画坐标轴上，后者则用于生成「模拟数据」。举例来说，当我们需要画一个函数的图像时，X 往往使用 linspace 生成，然后使用函数公式求得 Y，再 plot；当我们需要构造一些输入（比如 X）或中间输入（比如 Embedding、hidden state）时，random 会异常方便。

### 从 python 列表或元组创建

⭐⭐

重点掌握传入 list 创建一个 array 即可：`np.array(list)`

⚠️ 需要注意的是：「数据类型」。如果您足够仔细的话，可以发现下面第二组代码第 2 个数字是「小数」（注：Python 中 1. == 1.0），而 array 是要保证每个元素类型相同的，所以会帮您把 array 转为一个 float 的类型。

In [5]:
# 一个 list
np.array([1,2,3])

array([1, 2, 3])

In [6]:
# 二维（多维类似）
# 注意，有一个小数
np.array([[1, 2., 3], [4, 5, 6]])

array([[1., 2., 3.],
       [4., 5., 6.]])

In [7]:
# 你也可以指定数据类型
np.array([1, 2, 3], dtype=np.float16)

array([1., 2., 3.], dtype=float16)

In [8]:
# 如果指定了 dtype，输入的值都会被转为对应的类型，而且不会四舍五入
lst = [
    [1, 2, 3],
    [4, 5, 6.8]
]
np.array(lst, dtype=np.int32)

array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)

In [9]:
# 一个 tuple
np.array((1.1, 2.2))

array([1.1, 2.2])

In [10]:
# tuple，一般用 list 就好，不需要使用 tuple
np.array([(1.1, 2.2, 3.3), (4.4, 5.5, 6.6)])

array([[1.1, 2.2, 3.3],
       [4.4, 5.5, 6.6]])

In [11]:
# 转换而不是上面的创建，其实是类似的，无须过于纠结
np.asarray((1,2,3))

array([1, 2, 3])

In [12]:
np.asarray(([1., 2., 3.], (4., 5., 6.)))

array([[1., 2., 3.],
       [4., 5., 6.]])

## 使用 arange 生成

⭐⭐

range 是 Python 内置的整数序列生成器，arange 是 numpy 的，效果类似，会生成一维的向量。我们偶尔会需要使用这种方式来构造 array，比如：

需要创建一个连续一维向量作为输入（比如编码位置时可以使用）
需要观察筛选、抽样的结果时，有序的 array 一般更加容易观察
⚠️ 需要注意的是：在 reshape 时，目标的 shape 需要的元素数量一定要和原始的元素数量相等。

In [14]:
np.arange(12).reshape(3, 4)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [15]:
# 注意，是小数哦
np.arange(12.0).reshape(4, 3)

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.]])

In [16]:
np.arange(100, 124, 2).reshape(3, 2, 2)

array([[[100, 102],
        [104, 106]],

       [[108, 110],
        [112, 114]],

       [[116, 118],
        [120, 122]]])

In [17]:
# shape size 相乘要和生成的元素数量一致
np.arange(100., 124., 2).reshape(2,3,4)

ValueError: cannot reshape array of size 12 into shape (2,3,4)